In [ ]:
import os
from binascii import hexlify
from time import sleep
import datetime

In [ ]:
import tagcore

In [ ]:
import tagnet

In [ ]:
from tagdump import tagdump as td

In [ ]:
tagdir       = '/home/pi/tags/1fbcd99fd29f'
dblkdir      = tagdir+'/tag/sd/0/dblk' 
dblkbyte     = dblkdir+'/byte' 
dblkresync   = dblkdir+'/.resync'
dblklastsync = dblkdir+'/.last_sync'
dblkrecnum   = dblkdir+'/.recnum'

In [ ]:
!pwd

In [ ]:
def int32(x):
  if x>0xFFFFFFFF:
    raise OverflowError
  if x>0x7FFFFFFF:
    x=int(0x100000000-x)
    if x<2147483648:
      return -x
    else:
      return -2147483648
  return x

In [ ]:
def round_to_word(a):
    return ((a-(a%4)) if (a%4) else (a))

In [ ]:
def get_file_size(fn):
    ret=!stat  -c '%s' $fn
    offset=int(ret[0])
    int32(offset)
    return offset

In [ ]:
def err_by_value(val):
    for e in tagnet.tlv_errors:
        if e.value == val: break
    return e.name

In [ ]:
def size_to_err(pos):
    ipos = int32(pos)
    if ipos < 0:
        print('error', -ipos, err_by_value(-ipos))
        return -ipos
    return ipos

In [ ]:
def resync(fname, offset):
    !truncate -s $offset $fname
    limit = 1
    while (limit < 100):
        offset = get_file_size(dblkresync)
        if (offset):
            #print('=', limit, offset)
            return offset, limit
        limit += 1
        #print('.', limit, offset)
    return -1, 0

In [ ]:
def record_check(fn, offset):
    os.lseek(fn, offset, os.SEEK_SET)
    hdr=tagcore.core_headers.obj_dt_hdr()
    buf=bytearray(os.read(fn, len(hdr)))
    if len(buf) < len(hdr): return 0
    hdr.set(buf)
    rlen=hdr['len'].val
    rtype  = hdr['type'].val
    recnum = hdr['recnum'].val
    recsum = hdr['recsum'].val
    rt = hdr['rt']
    dlen = rlen - len(hdr)
    if (dlen > 0):
        buf.extend(os.read(fn, dlen))
    chksum = sum(bytearray(buf)[:rlen])
    chksum -= (recsum & 0xff00) >> 8
    chksum -= (recsum & 0x00ff)
    chksum &= 0xffff                # force to 16 bits vs. 16 bit recsum
    if (chksum != recsum):
        print('checksum error, expected: {:x}, got: {:x}'.format(chksum, recsum))
        print('.'.join(['{:02x}'.format(a) for a in buf[:20]]))
    else:
        dt = datetime.datetime(rt['year'].val, rt['mon'].val, rt['day'].val, rt['hr'].val, rt['min'].val, rt['sec'].val, rt['sub_sec'].val)
        print('{}: {}, len: {}, offset: {}, recnum: {}, recsum: {:x}/{:x}'.format(tagcore.dt_defs.dt_name(rtype),dt.isoformat(),rlen,offset,recnum,recsum,chksum))
        print('.'.join(['{:02x}'.format(a) for a in buf]))
    return round_to_word(rlen)

In [ ]:
def sync_check(fn, offset, display=False):
    os.lseek(fn, offset, os.SEEK_SET)
    sync=tagcore.core_headers.obj_dt_sync()
    buf=bytearray(os.read(fn, len(sync)))
    sync.set(buf)
    rlen   = sync['hdr']['len'].val
    rtype  = sync['hdr']['type'].val
    recnum = sync['hdr']['recnum'].val
    recsum = sync['hdr']['recsum'].val
    rt = sync['hdr']['rt']
    dt = datetime.datetime(rt['year'].val, rt['mon'].val, rt['day'].val, rt['hr'].val, rt['min'].val, rt['sec'].val, rt['sub_sec'].val)
    prev_sync = sync['prev_sync'].val
    dlen = rlen - len(sync)
    if (dlen > 0):
        buf.extend(os.read(fn, dlen))
    chksum = sum(bytearray(buf)[:rlen])
    chksum -= (recsum & 0xff00) >> 8
    chksum -= (recsum & 0x00ff)
    chksum &= 0xffff                # force to 16 bits vs. 16 bit recsum
    if (display):
        print('{}: {}, len: {}, offset/prev: {}/{}, recnum: {}, recsum: {:x}/{:x}'.format(
            tagcore.dt_defs.dt_name(rtype),
            dt.isoformat(),
            rlen,
            offset,
            prev_sync,
            recnum,
            recsum,
            chksum))
        print('.'.join(['{:02x}'.format(a) for a in buf]))
    if (chksum != recsum):
        print('checksum error, expected: {:x}, got: {:x}'.format(chksum, recsum))
    return prev_sync

# Find Tags

In [ ]:
!rm -f /home/pi/tags/.poll/found/*
!ls /home/pi/tags/.poll/new5

# Dblk Info

In [ ]:
dblk_fn=os.open(dblkbyte, os.O_RDWR)

In [ ]:
!ls -al $dblkbyte

In [ ]:
!ls -al $dblkdir/.recnum

In [ ]:
!ls -al $dblkrecnum

In [ ]:
!ls -al $dblkdir/.last_sync

In [ ]:
ret=!stat  -c '%s' $dblkdir/.last_sync
sync_offset=int(ret[0])
sync_offset

In [ ]:
record_check(dblk_fn, sync_offset)

In [ ]:
sync_check(dblk_fn, sync_offset)

# Find sync record

In [ ]:
STOP

In [ ]:
!ls -al $dblkdir/byte

In [ ]:
!truncate -s 190976 $dblkdir/.resync

In [ ]:
!truncate -s $sync_offset $dblkdir/.resync
!ls -al $dblkdir/.resync

In [ ]:
!ls -al $dblkdir/.resync

In [ ]:
ret=!stat  -c '%s' $dblkdir/.resync
resync_offset=int(ret[0])
int32(resync_offset)

In [ ]:
record_check(dblk_fn, resync_offset)

In [ ]:
sync_check(dblk_fn, resync_offset)

# Verify consecutive sync records

In [ ]:
ret=!stat  -c '%s' $dblklastsync
offset=int(ret[0])
int32(offset)

In [ ]:
#offset=get_file_size(dblkresync)
offset = 512
last_offset = 0
start=datetime.datetime.now()
print(start)
for i in range(2000):
    rtime = datetime.datetime.now()
    offset, limit = resync(dblkresync, offset)
    ttime = datetime.datetime.now() - rtime
    offset = int32(offset)
    if offset <= 0: break
    try:
        previous = sync_check(dblk_fn, offset)
        delta = offset - previous if () else 0
        print('*** {:4}[{:2}], {:4}bps, {:8} : {}'.format(i,
                                                          limit,
                                                          round((delta*8)/ttime.total_seconds(),2),
                                                          offset, previous))
        if (i) and (previous != last_offset): print('missed sync, previous: {}, last: {}'.format(previous, last_offset))
    except OSError:
        if offset: print(offset)
        sleep(1)
    last_offset = offset
    offset += 4
    #print('loop',offset)
print('total {} seconds, {} iterations, eof: {}, resync: {}, error: {}'.format(
                                                        round((datetime.datetime.now()-start).total_seconds(),1),
                                                        i,
                                                        get_file_size(dblkbyte),
                                                        get_file_size(dblkresync),
                                                        err_by_value(-offset)))

In [ ]:
sync_check(dblk_fn, 8864, display=True)

In [ ]:
offset = get_file_size(dblklastsync)
#offset = 460340
for i in range(100):
    if offset >= get_file_size(dblkbyte):
        sleep(1)
        print('offset: {}, eof: {}, last_sync: {}'.format(offset, get_file_size(dblkbyte), get_file_size(dblklastsync)))
        continue
    try:
        offset += record_check(dblk_fn, offset)
        print('*** count: {}, offset: {}, mod512: {}'.format(i, offset, offset % 512))
    except OSError:
        sleep(1)
        print('offset: {}, eof: {}, last_sync: {}'.format(offset, get_file_size(dblkbyte), get_file_size(dblklastsync)))
print('done')

In [ ]:
offset = get_file_size(dblklastsync)
while (True):
    offset = sync_check(dblk_fn, offset, display=True)    

In [ ]:
STOP

# Search for Sync records from file using tagdump

In [ ]:
!dd if=$dblkbyte of=tempdata status=progress

In [ ]:
infile  = td.TagFile(open('tempdata'))

In [ ]:
td.resync(infile, 0)

In [ ]:
offset = 0
while (True):
    print(offset)
    offset = td.resync(infile, offset)

# Search for Sync records from remote tag using tagdump

In [ ]:
infile  = td.TagFile(open(dblkbyte), net_io = True)

In [ ]:
td.resync(infile, 0)

In [ ]:
td.get_record(infile)

In [ ]:
offset = 0
while (True):
    print(offset)
    offset = td.resync(infile, offset) + 4
    rec = td.get_record(infile)
    print(rec)

In [ ]:
get_file_size(dblkresync)

In [ ]:
get_file_size(dblklastsync)

# Read records from remote tag using Tagdump

In [ ]:
infile  = td.TagFile(open(dblkbyte), net_io = True)

In [ ]:
td.resync(infile, 0)

In [ ]:
offset = 0
while (True):
    offset = size_to_err(get_file_size(dblkresync))
    print(offset)
    if (offset > get_file_size(dblkbyte)):
        break
    offset += 4
    offset = size_to_err(td.resync(infile, offset))
    print(td.get_record(infile))

# Other

In [ ]:
STOP

In [ ]:
i=4294967295
int32(i)

In [ ]:
int32()

In [ ]:
!ls

In [ ]:
help(tagcore)

In [ ]:
help(tagcore.core_headers)

In [ ]:
!ls /tmp